In [1]:
"""
Задание 1
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую 
принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, 
то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, 
то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.
"""

import pandas as pd
import numpy as np

#data = pd.read_csv('keywords2.csv') делал сет для наглядной проверки 
#также не вышло сделать перебор по ключам словаря geo_data - "Wrong number of items passed 3, placement implies 1"

data = pd.read_csv('keywords.csv')


def regional_tag(row):
    geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
        }
    return pd.Series(['Центр' if row['keyword'] in geo_data['Центр'] else 'Северо-Запад' if row['keyword'] in geo_data['Северо-Запад'] else 'Дальний Восток' if row['keyword'] in geo_data['Дальний Восток'] else 'unidentified' ])

data['region'] = data.apply(regional_tag, axis=1)

data.head(20)


,keyword,shows,region
0,вк,64292779,unidentified
1,одноклассники,63810309,unidentified
2,порно,41747114,unidentified
3,ютуб,39995567,unidentified
4,вконтакте,21014195,unidentified
5,одноклассники моя страница,19976138,unidentified
6,майл,15144673,unidentified
7,авито,14786593,unidentified
8,переводчик,14453469,unidentified
9,яндекс,13344734,unidentified


In [2]:
"""
Задание 2
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг
Результат классификации запишите в столбец class
"""

import pandas as pd
import numpy as np

ratings = pd.read_csv('ratings.csv')

def ratings_cl(row):  
        return pd.Series(['низкий рейтинг' if row['rating'] <= 2 else 'средний рейтинг' if 2 < row['rating'] <= 4 else 'высокий рейтинг'])

ratings['Class'] = ratings.apply(ratings_cl, axis=1)
ratings.head()


,userId,movieId,rating,timestamp,Class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


In [125]:
"""
Задание 3
Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). 
Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. 
Ответ дайте в днях.
"""
import pandas as pd
import numpy as np

data_m = pd.read_csv('ratings.csv')
a=0
kinomans_sum=0

data_n = data_m.drop(['movieId', 'rating'], axis=1)
data_n = data_n['userId'].value_counts().to_frame('count')
data_m = data_m.groupby('userId').agg(['min', 'max'])['timestamp']
data_m = data_m.drop(data_n.index[data_n['count'] <= 100].tolist())
max_sum = 0
min_sum = 0

for index, row in data_m.iterrows():
    max_sum += data_m['max'][index]
    min_sum += data_m['min'][index]
    a+=1

print('average cinephil lifetime =', (max_sum-min_sum)//(a*24*60*60), 'days')

average cinephil lifetime = 463 days


In [155]:
"""
Задание 4
Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". 
Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. 
Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, 
а пройдемся по следующему алгоритму: 

1. В переменную years запишите список из всех годов с 1950 по 2010."""

years = []
i = 1950
while i <= 2019:
    years.append(i)
    i += 1
    
"""2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. 
Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год"""

movies = pd.read_csv('movies.csv')
def production_year(row):  
    years_series = []
    counter = 0
    for ttl in movies['title']:
        counter = 0
        for yr in years:
            yr = str(yr)
            if yr in ttl and counter == 0:
                years_series.append(yr)
                counter +=1
        if counter == 0:
            years_series.append('1990')
    return years_series


"""3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'"""

movies['years'] = production_year(movies)
ratings = pd.read_csv('ratings.csv')
ratings = ratings.groupby('movieId').agg('mean')['rating'].to_frame()
ratings = ratings.join(movies.set_index('movieId'))

"""4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и 
отсортируйте результат по убыванию рейтинга"""

ratings = ratings.groupby('years').agg('mean')['rating'].to_frame()

ratings.sort_values('rating', ascending=False).head

<bound method NDFrame.head of          rating
years          
1951   3.906653
1952   3.802318
1962   3.772962
1953   3.738878
1957   3.693322
1973   3.688212
1971   3.643227
1969   3.604509
1972   3.585295
1958   3.571956
1977   3.561309
1970   3.559196
1976   3.555585
1990   3.552450
1964   3.541368
1960   3.521093
1965   3.517831
1954   3.499630
1955   3.494027
1978   3.480250
1963   3.476131
1974   3.474349
1959   3.451465
1961   3.447639
1950   3.436538
1981   3.433803
1984   3.421947
1982   3.389581
1979   3.385841
2007   3.380247
...         ...
1986   3.322234
1956   3.319404
2004   3.300785
1993   3.281731
1968   3.272118
1994   3.269216
1998   3.266041
2006   3.265219
1996   3.260178
1987   3.247274
1989   3.245381
1988   3.242909
2008   3.237304
1980   3.221179
1991   3.214035
2009   3.212944
1983   3.205512
1999   3.201568
2010   3.201324
2000   3.194736
2011   3.192694
2002   3.191918
1997   3.183169
2003   3.154081
2013   3.137302
2001   3.133080
2012   3.061191
2015   3.0